In [1]:
import skimage.io as io

In [2]:
cell_img = io.imread("./data/cell_3ch_patch.tif").astype(float)
cell_mask = io.imread("./data/mask.tif").astype(bool)

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib notebook

In [4]:
def line_array(color,i):
    import numpy as np
    if (color=="cyan") or (color=="c"):
        line = np.array([0/256, (i+1)/256, (i+1)/256, 1])
    elif (color=="magenta") or (color=="m"):
        line = np.array([(i+1)/256, 0/256, (i+1)/256, 1])
    elif (color=="blue") or (color=="b"):
        line = np.array([0/256, 0/256, (i+1)/256, 1])
    return(line)
def create_fluo_cm(color):
    from matplotlib import cm
    from matplotlib.colors import ListedColormap
    import numpy as np
    grays = cm.get_cmap('gray', 256)
    newcolors = grays(np.linspace(0, 1, 256))
    for i in range(1,newcolors.shape[0]):
        newcolors[i,:] = line_array(color,i)
    newcmp = ListedColormap(newcolors)
    return(newcmp)

In [5]:
cyan_cm = create_fluo_cm("c")
magenta_cm = create_fluo_cm("m")
cm_list = [magenta_cm, cyan_cm]

In [6]:
class IonChannel(object):
    def __init__(self,ion,lin,col,ide):
        self.id = ide
        self.x = int(col)
        self.y = int(lin)
        self.ion = ion
        self.state = 'closed'
    def ch_open(self):
        self.state = 'open'
    def ch_close(self):
        self.state = 'closed'

In [7]:
def create_channels(n=20,ion='K+'): 
    import skimage.measure as measure
    global cell_mask
    import numpy as np
    # get cell contour
    contours = measure.find_contours(cell_mask.astype(int),0)
    # calculate the step size necessary to divide the contours vector in 30 pieces
    step = len(contours[0])//20
    # calculate the rest that is left of the vector after dividing it in 30 pieces
    rest = len(contours[0])%20
    
    if ion=='K+':
        # takes the coordinates of each step
        coords = contours[0][::step]
        coords = coords[:-1]
    else:
        # takes the coordinates of each step, but starting at half step
        coords = contours[0][step//2::step] 
    
    ch_list = []
    coords = coords[np.random.choice(20, n, replace=False)]
    for counter,value in enumerate(coords):
        ch_list.append(IonChannel(ion,value[0],value[1],counter))
    return(ch_list)

In [8]:
# def create_ch_carousel(ch_list,ion):
#     if ion=='K+':
#         border = '2px solid yellow'
#     elif ion=='Na+':
#         border = '2px solid white'
#     item_layout = widgets.Layout(height='30px', min_width='40px',align_self='flex-start',border = border)
#     items = [widgets.ToggleButton(layout=item_layout, description=str(i), button_style='danger',tooltip=ion) for i in range(len(ch_list))]
#     box_layout = widgets.Layout(overflow='scroll hidden',
#                     border='1px solid black',
#                     width='400px',
#                     height='',
#                     flex_flow='row',
#                     display='flex',
#                     margin = '10px'
#                     )
#     carousel = widgets.Box(children=items, layout=box_layout)
#     return(carousel)

In [98]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np

'''################ Callback functions ################'''
# Updates image from channel selection change
def update_screen(v):
    '''Function that updates images when images need to be update 
    (for example, when channel is changed)'''
    global cell_img, K_list, Na_list
    ax.clear()
    # clear images
    out.clear_output(wait=True)
    with out:
        # if channel 1 is selected (index=0)
        if channel_sel.index==0:
            # show cell image
            ax.imshow(cell_img[:,:,0],cmap='gray')
            ax.axis('off')
        # if channel 2 is selected (index=1)
        else:
            K_open_count = 0
            K_closed_count = 0
            Na_open_count = 0
            Na_closed_count = 0
            # show K image
            ax.imshow(cell_img[:,:,channel_sel.index],cmap=cm_list[channel_sel.index-1],vmin=0,vmax=255)
            ax.axis('off')
            if adv_check.value==True:
                for ch in K_list:
                    if ch.state=='open':
                        color = 'lightgreen'
                    else:
                        color = 'r'
                    offset = (-12,5)
                    if ch.id>9:
                        offset = (-18,5)
                    if ((ch.state=='open') & (K_open_count==0)):
                        marker_K_open, = ax.plot(ch.x,ch.y,marker='D',color=color,ms=12,mew=2,mec='yellow',ls='')
                        marker_K_open.set_label('Canal de K+ aberto')
                        K_open_count += 1
                    if ((ch.state=='closed') & (K_closed_count==0)):
                        marker_K_closed, = ax.plot(ch.x,ch.y,marker='D',color=color,ms=12,mew=2,mec='yellow',ls='')
                        marker_K_closed.set_label('Canal de K+ fechado')
                        K_closed_count += 1
                    ax.plot(ch.x,ch.y,marker='D',color=color,ms=12,mew=2,mec='yellow')
#                     ax.annotate(text=ch.id,xy=(ch.x,ch.y),xytext=offset,textcoords='offset pixels',color='yellow',fontsize='large')
                for ch in Na_list:
                    if ch.state=='open':
                        color = 'lightgreen'
                    else:
                        color = 'r'
                    offset = (-15,5)
                    if ch.id>9:
                        offset = (-20,5)
                    if ((ch.state=='open') & (Na_open_count==0)):
                        marker_Na_open, = ax.plot(ch.x,ch.y,marker='s',color=color,ms=12,mew=2,mec='white',ls='')
                        marker_Na_open.set_label('Canal de Na+ aberto')
                        Na_open_count += 1
                    if ((ch.state=='closed') & (Na_closed_count==0)):
                        marker_Na_closed, = ax.plot(ch.x,ch.y,marker='s',color=color,ms=12,mew=2,mec='white',ls='')
                        marker_Na_closed.set_label('Canal de Na+ fechado')
                        Na_closed_count += 1
                    ax.plot(ch.x,ch.y,marker='s',color=color,ms=12,mew=2,mec='white')
#                     ax.annotate(text=ch.id,xy=(ch.x,ch.y),xytext=offset,textcoords='offset pixels',color='white',fontsize='large')
                ax.legend()
        display(fig)
        
# Updates graphics
def update_graphics(Na_o, Na_i, K_o, K_i):
    '''Function that updates graphics'''
    ax2[0].clear()
    ax2[1].clear()
    # clear graphics
    out2.clear_output(wait=True)
    with out2:
        # show cell image on axes
        time = np.linspace(0,100)
        #sources: https://www.cell.com/biophysj/pdf/S0006-3495(81)84820-5.pdf
        #        https://en.wikipedia.org/wiki/Goldman_equation
        # approximating R*T/F to 26.7 mV (at 37ºC)
        P_K = 1.33e-7 #m/s
        P_Na = 1.92e-10 #m/s
        Em = 26.7*np.log( ( (P_Na * Na_o) + (P_K * K_o) ) / ( (P_Na * Na_i) + (P_K * K_i) ) )
#         v0 = -80
        potential = Em*np.ones(time.shape)
        ax2[0].plot(time, potential)
        ax2[0].set_ylabel('Potencial de Membrana (mV)')
        ax2[0].set_xlabel('Tempo (s)')
        ax2[0].set_ylim([-210,210])
        
        
        print('Potencial = ' + str(np.around(Em,2)) + ' mV')
        Na_i_vector = Na_i*np.ones(time.shape)
        Na_o_vector = Na_o*np.ones(time.shape)
        K_i_vector = K_i*np.ones(time.shape)
        K_o_vector = K_o*np.ones(time.shape)
        ax2[1].plot(time, Na_i_vector,'cyan',label='[Na+]i')
        ax2[1].plot(time, Na_o_vector,'b',label='[Na+]o')
        ax2[1].plot(time, K_i_vector,'magenta',label='[K+]i')
        ax2[1].plot(time, K_o_vector,'purple',label='[K+]o')
        ax2[1].set_ylabel('Concentração (mM)')
        ax2[1].set_xlabel('Tempo (s)')
        ax2[1].legend(loc='upper right',fontsize=14)
        display(fig2)
        
def on_concentration_slider(v):
    '''Function that updates images when concentration bars are changed'''
    from skimage.filters import gaussian
    import numpy as np
    global cell_img, cell_mask, Na_o, Na_i, K_o, K_i
    value = v['new']
    if value==0:
        value = 0.1
    # if changed bar is K+ i
    if v['owner'].description_tooltip.startswith('[K+]i'):
        cell_img[:,:,1][cell_mask] = v['new']
        cell_img[:,:,1] = gaussian(cell_img[:,:,1],sigma=2,multichannel=False)
        channel_sel.index = 1
        K_i = value
    # if changed bar is K+ o
    elif v['owner'].description_tooltip.startswith('[K+]o'):
        cell_img[:,:,1][np.invert(cell_mask)] = v['new']
        cell_img[:,:,1] = gaussian(cell_img[:,:,1],sigma=2,multichannel=False)
        channel_sel.index = 1
        K_o = value
    # if changed bar is Na+ i
    elif v['owner'].description_tooltip.startswith('[Na+]i'):
        cell_img[:,:,2][cell_mask] = v['new']
        cell_img[:,:,2] = gaussian(cell_img[:,:,2],sigma=2,multichannel=False)
        channel_sel.index = 2
        Na_i = value
    # if changed bar is Na+ i
    elif v['owner'].description_tooltip.startswith('[Na+]o'):
        cell_img[:,:,2][np.invert(cell_mask)] = v['new']
        cell_img[:,:,2] = gaussian(cell_img[:,:,2],sigma=2,multichannel=False)
        channel_sel.index = 2
        Na_o = value
    #Update screen
    update_screen(0)        
    update_graphics(Na_o, Na_i, K_o, K_i) 

def on_adv_check(v):
#     global Na_o, Na_i, K_o, K_i, Na_o_slider
    if v['new']==True:
        hbox_num_ch.children = [ch_label, hbox_K_ch_slider, hbox_Na_ch_slider]
        if K_ch_slider.value==1:
            label_K_ch_slider.value = 'canal aberto'
        else:
            label_K_ch_slider.value = 'canais abertos'
        if Na_ch_slider.value==1:
            label_Na_ch_slider.value = 'canal aberto'
        else:
            label_Na_ch_slider.value = 'canais abertos'
        hbox_num_ch.layout.visibility='visible'
#         hbox_ch_carousel.children = [K_carousel, Na_carousel]
#         hbox_ch_carousel.layout.visibility='visible'  
        Na_o_slider.value = 140
        Na_i_slider.value = 10
        K_o_slider.value = 5
        K_i_slider.value = 140
        Na_o_slider.disabled = True
        Na_i_slider.disabled = True
        K_o_slider.disabled = True
        K_i_slider.disabled = True
        channel_sel.index = 1
    else:
        hbox_num_ch.children = []
        hbox_num_ch.layout.visibility='hidden'
#         hbox_ch_carousel.children = []
#         hbox_ch_carousel.layout.visibility='hidden'
        Na_o_slider.disabled = False
        Na_i_slider.disabled = False
        K_o_slider.disabled = False
        K_i_slider.disabled = False
    update_screen(0)
    
# def on_tbutton(v):
#     global K_list, Na_list
#     ion = v['owner'].tooltip
#     value = v['new']
#     ch_id = int(v['owner'].description)
#     if value==True:
#         v['owner'].button_style='success'
#         if ion=='K+':
#             K_list[ch_id].ch_open()
#         elif ion=='Na+':
#             Na_list[ch_id].ch_open()
#     else:
#         v['owner'].button_style='danger'
#         if ion=='K+':
#             K_list[ch_id].ch_close()
#         elif ion=='Na+':
#             Na_list[ch_id].ch_close()
#     update_screen(0)
    
def on_channel_slider(v):
    global K_list, Na_list  #,K_carousel,Na_carousel
    value = v['new']
    if v['owner'].description.startswith('K+'):
        if K_ch_slider.value==1:
            label_K_ch_slider.value = 'canal aberto'
        else:
            label_K_ch_slider.value = 'canais abertos'
        for i in range(value):
            K_list[i].ch_open()
        for i in range(value,len(K_list)):
            K_list[i].ch_close()
#         K_list = create_channels(n=v['new'],ion='K+')
#         K_carousel = create_ch_carousel(K_list,ion='K+')
#         for button in K_carousel.children:
#             button.observe(on_tbutton, 'value')
#         channel_sel.index = 1
    elif v['owner'].description.startswith('Na+'):
        if Na_ch_slider.value==1:
            label_Na_ch_slider.value = 'canal aberto'
        else:
            label_Na_ch_slider.value = 'canais abertos'
        for i in range(value):
            Na_list[i].ch_open()
        for i in range(value,len(Na_list)):
            Na_list[i].ch_close()
#         Na_list = create_channels(n=v['new'],ion='Na+')
#         Na_carousel = create_ch_carousel(Na_list,ion='Na+')
#         for button in Na_carousel.children:
#             button.observe(on_tbutton, 'value')
#         channel_sel.index = 2
#     hbox_ch_carousel.children = [K_carousel, Na_carousel]
    update_screen(0)
        
'''################ Widgets construction ################'''
# Title widget for radiobox for channel selection
label_channel_sel = widgets.HTML(value = f"<b><font color='black'>{'Seletor de Tela:'}</b>")
# Widget of radiobox for channel selection
channel_sel = widgets.RadioButtons(
    options=['Tela da célula', 'Tela do potássio [K+]', 'Tela do sódio [Na+]'],
    description='',
    disabled=False 
)

# widget of output images
out = widgets.Output(layout={'border': '1px solid black'})
# widget of output graphic
out2 = widgets.Output(layout={'border': '1px solid black'})
# widget label for concentration sliders
label_concentration = widgets.Label('Arraste as barras para alterar as concentrações dos íons fora e dentro da célula:')

label_Na_o = widgets.HTML(value = f"<b><font color='blue'>{'[Na+]fora (mM):'}</b>")
label_Na_o.layout.width = '112px'
# widget of Na+ o slider
Na_o_slider = widgets.IntSlider(
    value=140,
    min=0,
    max=200,
    step=1,
    description='',
    description_tooltip = '[Na+]o (mM)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
hbox_Na_o_slider = widgets.HBox([label_Na_o,Na_o_slider])
# hbox_Na_o_slider.layout.border = '1px solid black'


label_Na_i = widgets.HTML(value = f"<b><font color='43DDCF'>{'[Na+]dentro (mM):'}</b>")
# widget of Na+ i slider
Na_i_slider = widgets.IntSlider(
    value=10,
    min=0,
    max=200,
    step=1,
    description='',
    description_tooltip='[Na+]i (mM)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
hbox_Na_i_slider = widgets.HBox([label_Na_i,Na_i_slider])
# hbox_Na_i_slider.layout.border = '1px solid black'

label_K_o = widgets.HTML(value = f"<b><font color='purple'>{'[K+]fora (mM):'}</b>")
label_K_o.layout.width = '105px'
# widget of K+ o slider
K_o_slider = widgets.IntSlider(
    value=5,
    min=0,
    max=200,
    step=1,
    description='',
    description_tooltip='[K+]o (mM)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
hbox_K_o_slider = widgets.HBox([label_K_o,K_o_slider])
# hbox_K_o_slider.layout.border = '1px solid black'

label_K_i = widgets.HTML(value = f"<b><font color='magenta'>{'[K+]dentro (mM):'}</b>")
# widget of K+ o slider
K_i_slider = widgets.IntSlider(
    value=140,
    min=0,
    max=200,
    step=1,
    description='',
    description_tooltip='[K+]i (mM)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
hbox_K_i_slider = widgets.HBox([label_K_i,K_i_slider])
# hbox_K_i_slider.layout.border = '1px solid black'

adv_check = widgets.Checkbox(
    value=False,
    description='',
    disabled=False,
    indent=False
)
# adv_check.layout.border = '1px solid black'
# adv_check.layout.display = 'flex'
adv_check.layout.width='20px'
label_adv_check = widgets.Label('Modo Avançado: brinque de abrir e fechar os canais manualmente!')
# label_adv_check.layout.display = 'flex'
# label_adv_check.layout.justify_content = 'flex-start'
adv_box = widgets.HBox([adv_check,label_adv_check])
# adv_box.layout.align_items = 'flex-start'
# adv_box.layout.justify_content = 'flex-start'
# adv_box.layout.align_self = 'flex-start'
# adv_check.style.description_width = 'initial'
ch_label = widgets.Label(
    value="Escolha número de canais abertos de:"
)

Na_ch_slider = widgets.IntSlider(
    value=1,
    min=0,
    max=20,
    step=1,
    description='Na+ (■)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
label_Na_ch_slider = widgets.Label(
    value="canais abertos"
)
hbox_Na_ch_slider = widgets.HBox([Na_ch_slider,label_Na_ch_slider])

K_ch_slider = widgets.IntSlider(
    value=20,
    min=0,
    max=20,
    step=1,
    description='K+ (♦)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(align_self='flex-start')
)
label_K_ch_slider = widgets.Label(
    value="canais abertos"
)
hbox_K_ch_slider = widgets.HBox([K_ch_slider,label_K_ch_slider])

K_list = create_channels(n=20,ion='K+')
for i in range(K_ch_slider.value):
    K_list[i].ch_open()
# K_carousel = create_ch_carousel(K_list,ion='K+')
# for button in K_carousel.children:
#     button.observe(on_tbutton, 'value')
Na_list = create_channels(n=20,ion='Na+')
for i in range(Na_ch_slider.value):
    Na_list[i].ch_open()
# Na_carousel = create_ch_carousel(Na_list,ion='Na+')
# for button in Na_carousel.children:
#     button.observe(on_tbutton, 'value')

Na_o = Na_o_slider.value
Na_i = Na_i_slider.value
K_o = K_o_slider.value
K_i = K_i_slider.value

# Block of widgets containing Na+ sliders
vbox_conc_Na = widgets.VBox([hbox_Na_o_slider,hbox_Na_i_slider],justify_content='center')
# Block of widgets containing K+ sliders
vbox_conc_K = widgets.VBox([hbox_K_o_slider,hbox_K_i_slider],justify_content='center')
# Block of widgets containing num K+ channel slider
hbox_num_ch = widgets.VBox(justify_content='center')
hbox_num_ch.layout.visibility = 'hidden'
hbox_ch_carousel = widgets.HBox()
hbox_ch_carousel.layout.visibility='hidden'

# Block of widgets containing all concentration sliders
hbox_ions = widgets.HBox([vbox_conc_K,vbox_conc_Na],justify_content='center',align_content='center')
# Block of widgets containing channel selection and output images
vbox_fig = widgets.VBox([label_channel_sel,channel_sel,out,label_concentration,hbox_ions,adv_box,hbox_num_ch,out2], justify_content='center')
# Show cell image on first time
with out:
    # create figure and axes
    fig, ax = plt.subplots(figsize=(15,10))
    # show cell image on axes
    ax.imshow(cell_img[:,:,0],cmap='gray')
    ax.axis('off')
    # draw changes
    plt.show(fig)
out.layout.display = 'flex'
out.layout.align_items = 'center'
out.layout.justify_items = 'center'
out.layout.align_self = 'center'

# Show graphics on first time
with out2:
    # create figure and axes
    fig2, ax2 = plt.subplots(1,2,figsize=(15,5))
    # show cell image on axes
    time = np.linspace(0,100)
    P_K = 1.33e-7 #m/s
    P_Na = 1.92e-10 #m/s
    Em = 26.7*np.log( ( (P_Na * Na_o) + (P_K * K_o) ) / ( (P_Na * Na_i) + (P_K * K_i) ) )
    potential = Em*np.ones(time.shape)
    ax2[0].plot(time, potential)
    ax2[0].set_ylabel('Potencial de Membrana (mV)', size=16)
    ax2[0].set_xlabel('Tempo (s)',size=16)
    ax2[0].set_ylim([-210,210])
    ax2[0].tick_params(labelsize=16)
    
    Na_i_vector = Na_i*np.ones(time.shape)
    Na_o_vector = Na_o*np.ones(time.shape)
    K_i_vector = K_i*np.ones(time.shape)
    K_o_vector = K_o*np.ones(time.shape)
    ax2[1].plot(time, Na_i_vector,'cyan',label='[Na+]dentro')
    ax2[1].plot(time, Na_o_vector,'b',label='[Na+]fora')
    ax2[1].plot(time, K_i_vector,'magenta',label='[K+]dentro')
    ax2[1].plot(time, K_o_vector,'purple',label='[K+]fora')
    ax2[1].set_ylabel('Concentração (mM)',size=16)
    ax2[1].set_xlabel('Tempo (s)',size=16)
    ax2[1].legend(loc='upper right',fontsize=14)
    ax2[1].tick_params(labelsize=16)
    # draw changes
    plt.show(fig2)

# Link changes in Radiobox widget (channel selection) to screen update function
channel_sel.observe(update_screen,names='value')
# Link changes in slider widgets (ion concentrations) to concentration function
K_i_slider.observe(on_concentration_slider, names='value')
K_o_slider.observe(on_concentration_slider, names='value')
Na_i_slider.observe(on_concentration_slider, names='value')
Na_o_slider.observe(on_concentration_slider, names='value')
adv_check.observe(on_adv_check, names='value')
K_ch_slider.observe(on_channel_slider, names='value')
Na_ch_slider.observe(on_channel_slider, names='value')

vbox_fig_layout = widgets.Layout()
vbox_fig_layout.margin = '20px'
vbox_fig.layout = vbox_fig_layout

main_box = widgets.VBox([vbox_fig])
main_box.layout.align_items = 'center'
main_box.layout.border = '1px solid black'
main_box.box_style = ''

# Display main Block of widgets
main_box